# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [13]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [14]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/andi/Documents/projects/UD-DEND-Data-Modeling-with-Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [15]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print("length of full_data_rows_list: {0}".format(len(full_data_rows_list)))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


length of full_data_rows_list: 8056


In [16]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. We will complete the Apache Cassandra coding portion of the project. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [17]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [18]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [19]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [20]:
# define a pandas row factory to easily convert a cassandra query to a pandas dataframe
def pandas_factory(colnames, rows):
    """
    Description: This function defines a pandas row factory and helps 
                 to easily convert a cassandra query to a pandas dataframe

    Arguments:
        colnames: column names for the pandas dataframe
        rows: rows for the pandas dataframe

    Returns:
        pandas dataframe
    """

    return pd.DataFrame(rows, columns=colnames)

# config the cassandra session to the defined pandas row factory
session.row_factory = pandas_factory
session.default_fetch_size = None

## We will create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Task 1: Getting the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
For this query sessionId and itemInSession are considered as the composite key, because we need to include  
these two columns into the WHERE-clause to search for the specific sessionId and itemInSession as mentioned above.  
I have chosen also the columns artist text, song_title text and song_length because we need these columns for the desired output.

In [21]:
# Creating the table song_info_session for the first query as described above
query = "CREATE TABLE IF NOT EXISTS song_info_session "
query = query + "(session_id int, item_in_session int, artist text, song_title text, song_length float, "
query = query + "PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [22]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

# Insert the relevant data to the table song_info_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Create the INSERT statement for this task
        query = "INSERT INTO song_info_session (session_id, item_in_session, artist, song_title, song_length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [23]:
## Create the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song_title, song_length "
query = query + "FROM song_info_session "
query = query + "WHERE session_id=338 AND item_in_session=4"

try:
    result = session.execute(query)
except Exception as e:
    print(e)

df = result._current_rows
df.head()

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Task 2: Getting only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
For this query i used user_id and session_id as a composite partition key because we have to include this in the WHERE statement.  
Also i choosed the item_in_session-Column as a clustering key because we need to sort by them.  
Then we need also the columns song_title, artist, first_name and last_name becaue we need these columns for the desired output.

In [24]:
# Creating table song_playlist_session for the first query as described above
query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + "(user_id int, session_id int, item_in_session int, song_title text, artist text, first_name text, last_name text, "
query = query + "PRIMARY KEY ((user_id, session_id), item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [25]:
# Insert the relevant data to the table song_playlist_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Create the INSERT statement for this task
        query = "INSERT INTO song_playlist_session (user_id, session_id, item_in_session, song_title, artist, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[9], line[0], line[1], line[4]))

In [26]:
# SELECT statement to verify the data was entered into the table
query = "SELECT artist, song_title, first_name, last_name "
query = query + "FROM song_playlist_session "
query = query + "WHERE user_id=10 AND session_id=182"

try:
    result = session.execute(query)
except Exception as e:
    print(e)

df = result._current_rows
df.head()

,artist,song_title,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Task 3: Getting every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
For this query, I used song_title as the partition key and user_id as the clustering key because we have to use the song_title in the  
WHERE clause and the user_id we need to make sure the uniqueness for each entry in the table and for the query we can make sure we catch each user.

In [27]:
# Creating table user_info_song for the first query as described above
query = "CREATE TABLE IF NOT EXISTS user_info_song "
query = query + "(song_title text, user_id int, first_name text, last_name text, "
query = query + "PRIMARY KEY (song_title, user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [28]:
# Insert the relevant data to the table user_info_song
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Create the INSERT statement for this task
        query = "INSERT INTO user_info_song (song_title, user_id, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [29]:
# SELECT statement to verify the data was entered into the table
query = "SELECT first_name, last_name "
query = query + "FROM user_info_song "
query = query + "WHERE song_title='All Hands Against His Own'"

try:
    result = session.execute(query)
except Exception as e:
    print(e)

df = result._current_rows
df.head()

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Clean-up tasks

In [30]:
# Drop the table before closing out the sessions
table_names = ["song_info_session", "song_playlist_session", "user_info_song"]

for table in table_names:
    query = "DROP TABLE {0}".format(table)

    try:
        session.execute(query)
    except Exception as e:
        print(e)

In [31]:
# Closing the session
session.shutdown()
# Shutdown the cluster
cluster.shutdown()